In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
df = pd.read_csv('stud.csv')
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [3]:
X=df.drop(columns='math_score',axis=1)
X

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75
...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,99,95
996,male,group C,high school,free/reduced,none,55,55
997,female,group C,high school,free/reduced,completed,71,65
998,female,group D,some college,standard,completed,78,77


In [4]:
y=df['math_score']
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math_score, Length: 1000, dtype: int64

In [9]:
# numerical and categorical features 
numerical_feature = [feature for feature in X.columns if X[feature].dtype!='O']
categorical_feature = [feature for feature in X.columns if X[feature].dtype=='O']
numerical_feature = X.select_dtypes(exclude='object').columns
categorical_feature = X.select_dtypes(include='object').columns
numerical_feature,categorical_feature

(Index(['reading_score', 'writing_score'], dtype='object'),
 Index(['gender', 'race_ethnicity', 'parental_level_of_education', 'lunch',
        'test_preparation_course'],
       dtype='object'))

In [12]:
# create column transformers with 3 type of transformers
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()
preprocessor = ColumnTransformer(
    [
        ('OneHotEncoder',categorical_transformer,categorical_feature),
        ("StandardScaler",numeric_transformer,numerical_feature)
    ]
)


In [13]:
X=preprocessor.fit_transform(X)

In [14]:
# train test split 
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


In [16]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,root_mean_squared_error,r2_score

def evaluate_model(true,predicted):
    mae =mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = root_mean_squared_error(true,predicted)
    r2_square = r2_score(true,predicted)
    return mae,mse,rmse,r2_square

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
# from xgboost import XGBRegressor
from catboost import CatBoostRegressor

models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(),
    "DecisionTreeRegressor": DecisionTreeRegressor(),
    "RandomForestRegressor": RandomForestRegressor(),
    "GradientBoostingRegressor": GradientBoostingRegressor(),
    "KNeighborsRegressor": KNeighborsRegressor(),
    "SVR": SVR(),
    # "XGBRegressor": XGBRegressor(),
    "CatBoostRegressor": CatBoostRegressor(verbose=False)
}
    # return models


In [20]:
model_list=[]
r2_list=[]
max_r2_model_score=-10000
best_model = None
for model_name,model in models.items():
    model_list.append(model_name)
    model.fit(X_train,y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    mae_train,mse_train,rmse_train,r2_square_train = evaluate_model(y_train,y_pred_train)
    mae_test,mse_test,rmse_test,r2_square_test = evaluate_model(y_test,y_pred_test)
    print("Model Name",model_name)
    print(f'r2_score train',r2_square_train)
    if r2_square_test> max_r2_model_score:
        max_r2_model_score = r2_square_test
        best_model = model_name
    print(f'r2_score test',r2_square_test)

print("BEST MODEL",best_model)


Model Name LinearRegression
r2_score train 0.8743172040139593
r2_score test 0.8804332983749565
Model Name Ridge
r2_score train 0.8743042615212909
r2_score test 0.8805931485028737
Model Name Lasso
r2_score train 0.8071462015863456
r2_score test 0.8253197323627853
Model Name ElasticNet
r2_score train 0.7337786319564656
r2_score test 0.7396242583916663
Model Name DecisionTreeRegressor
r2_score train 0.9996534669718089
r2_score test 0.7466285633504617
Model Name RandomForestRegressor
r2_score train 0.9766317024794028
r2_score test 0.8466606549741522
Model Name GradientBoostingRegressor
r2_score train 0.9050396644022572
r2_score test 0.8721717543729617
Model Name KNeighborsRegressor
r2_score train 0.8554279731066252
r2_score test 0.7834973847119895
Model Name SVR
r2_score train 0.8081281585902299
r2_score test 0.7286001513223705
Model Name XGBRegressor
r2_score train 0.9954995512962341
r2_score test 0.8277965784072876
Model Name CatBoostRegressor
r2_score train 0.9589358676277713
r2_score t